## threading 的多线程并发
- 对比多进程并发： 
  - 优点 ： 资源消耗少
  - 缺点 ： 需要注意共享该资源的争夺  (注意GIL问题)
- 实现步骤
  1. 创建套接字，绑定，监听
  2. 接收客户端请求，创建新的线程
  3. 主线程继续等待其他客户端连接，分支线程执行客户    端请求
  4. 处理完客户端请求后，分支线程退出，关闭客户端套    接字

In [ ]:
# thread_server
from socket import * 
from threading import * 
import sys

HOST = '0.0.0.0'
PORT = 8888
ADDR = (HOST,PORT)

#创建套接字
s = socket()
s.bind(ADDR)
s.listen(5)

#客户端处理函数
def handler(connfd):
    print("Got connection from",connfd.getpeername())
    while True:
        data = connfd.recv(1024).decode()
        if not data:
            break
        connfd.send(b"receive your message")
    connfd.close()


while True:
    try:
        connfd,addr = s.accept()
    except KeyboardInterrupt:
        s.close()
        sys.exit("服务器退出")
    except Exception as e:
        print(e)
        continue 

    t = Thread(target = handler,args = (connfd,))
    t.setDaemon(True)
    t.start()


## socket 服务器继承模块
- python2 SocketServer
- python3 socketserver

- 功能:通过模块的接口完成基于多进程/多线程的tcp/udp 的socket并发程序

```python

#模块类
import socketserver
dir(socketserver)

DatagramRequestHandler  处理udp请求 
StreamRequestHandler    处理tcp请求

UDPServer   提供udp服务端类   
TCPServer   提供tcp服务端类

ForkingMixIn   提供进程创建
ForkingTCPServer   ==》ForkingMixIn + TCPServer
ForkingUDPServer   ==》ForkingMixIn + UDPServer

ThreadingMixIn  提供线程创建
ThreadingTCPServer  ==》ThreadingMixIn + TCPServer
ThreadingUDPServer  ==》ThreadingMixIn + TCPServer

```

In [2]:
from socketserver import * 

# 创建多进程  tcp 并发

# 创建进程tcp服务器类
# class Server(ForkingTCPServer):
# class Server(ForkingMixIn,TCPServer):
    # pass 

#多线程tcp并发
class Server(ThreadingTCPServer):
    pass

#具体请求处理类
class Handler(StreamRequestHandler):
    def handle(self):
        #self.request 相当于accept创建的套接字
        print("Connect from",self.request.getpeername())
        while True:
            data = self.request.recv(1024).decode()
            if not data:
                break
            print(data)
            self.request.send(b"Receive your message")

#生产服务器对象,传入addr和具体处理类
server = Server(("0.0.0.0",8888),Handler)

#启动服务器
server.serve_forever()

In [ ]:
from socketserver import * 

#多进程 udp并发
class Server(ForkingMixIn,UDPServer):
    pass 

class Handler(DatagramRequestHandler):
    def handle(self):
        while True:
            #接收消息
            data = self.rfile.readline().decode()
            if not data:
                break
            print(data)
            #发送消息
            self.wfile.write\
            (b"Receive your message")

server = Server(('127.0.0.1',8888),Handler)
server.serve_forever()

## 基于多线程并发的 HTTPServer
- HTTPServer的作用 
  - 1.就收浏览器发送的http请求
  - 2.对http请求进行解析
  - 3.组织响应内容进行回发

- 升级
  * 使用多线程的并发可以同时处理多个客户端请求
  * 增加简单的应用程序，使浏览器不仅可以访问静态网页   也可以访问后台程序
  * 使用类对服务器功能模块进行封装

- 技术点 ： 
  - threading 并发 
  - tcp socket 套接字
  - http协议响应内容进行字符串拼接

In [ ]:
'''
http server 第二版
'''
from socket import *  
from threading import Thread 
import time 

#存放静态页面
STATIC_DIR = './static'
ADDR = ('0.0.0.0',8000)

#httpserver类　封装服务器功能
class HTTPServer(object):
    def __init__(self,addr):
        #套接字创建
        self.sockfd = socket()
        self.sockfd.setsockopt\
        (SOL_SOCKET,SO_REUSEADDR,1) 
        self.sockfd.bind(addr)
        self.sockfd.listen(5)
        #为对象添加属性
        self.name = "HTTPServer"
        self.port = addr[1]
        self.address = addr 

    #　监听客户端链接请求，创建新的线程处理
    def serve_forever(self):
        print("Listen to port %d..."%self.port)
        while True:
            connfd,addr = self.sockfd.accept()
            #创建新的线程处理具体请求
            clientThread = Thread\
            (target = self.handleRequest,args = (connfd,))
            clientThread.setDaemon(True)
            clientThread.start()

    def handleRequest(self,connfd):
        #接受客户端请求
        request = connfd.recv(4096)
        #解析请求
        requestHeadlers = request.splitlines()
        #打印请求行
        print(connfd.getpeername(),':',\
            requestHeadlers[0])

        #获取具体请求内容
        getRequest = \
        str(requestHeadlers[0]).split(' ')[1]

        if getRequest == '/' or getRequest[-5:] == ".html":
            self.get_html(connfd,getRequest)
        else:
            self.get_data(connfd,getRequest)        
        connfd.close()

    def get_html(self,connfd,page):
        if page == '/':
            getFilename = STATIC_DIR + "/index.html"
        else:
            getFilename = STATIC_DIR + page
        
        try:
            f = open(getFilename)
        except Exception:
            #没有找到页面
            responseHeaders = "HTTP/1.1 404 not found\r\n"
            responseHeaders += "\r\n"
            responseBody = "===Sorry,the page not found==="
        else:
            responseHeaders = "HTTP/1.1 200 OK\r\n"
            responseHeaders += "\r\n" 
            responseBody = f.read()
        finally:
            response = responseHeaders + responseBody
            connfd.send(response.encode())

    def get_data(self,connfd,data):
        responseHeaders = 'HTTP/1.1 200 OK\r\n'
        responseHeaders += '\r\n'

        if data == "/time":
            responseBody = time.ctime()
        elif data == "/tedu":
            responseBody = "1234567"
        else:
            responseBody = "the data not found"

        response = responseHeaders + responseBody
        connfd.send(response.encode())



if __name__ == "__main__":
    #生成对象
    httpd = HTTPServer(ADDR)
    #启动服务器
    httpd.serve_forever()